# Random number generation (Mersenne Twister)

-------------------------

_**Problem Instructions:**_

**a) Research on MT19937 Algorithm**

Research the MT19937 (Mersenne Twister) algorithm for generating uniform random numbers. Provide a discussion on its features, period length, and efficiency compared to other random number generators such as linear congruential generators (LCG) and hardware-based generators. Consider discussing real-world applications where MT19937 is commonly used and any potential limitations it may have.

**b) Distribution Sampling**

Using the MT19937 algorithm, write a program that generates N samples from each of the following continuous distributions:
- Uniform(a, b)
- Exponential(λ)
- Normal(μ, σ²)
- Any other continuous distribution not discussed in class (e.g. Beta, Cauchy, Pareto, etc)

Explain the process of transforming uniform random numbers into these distributions. Provide clear mathematical justifications for the transformations used (e.g., inverse transform sampling, Box-Muller method for normal distribution). Plot the histograms of your generated samples and compare them with theoretical probability density functions (PDFs). Discuss any discrepancies and potential reasons for deviations.

-------------------------

## Understanding of the Problem

### A) Research on MT19337 Algorithm

For this part, the following tasks are expected to be done:
- Understand the Mersenne Twister algorithm -- a pseudorandom number generator (PRNG)
- Discuss the features of the MT19937 algorithm.
    - Discuss what is period length and what it is for Mersenne Twister algorithm.
    - Discuss the efficiency in comparison to:
        - Linear Congruential Generators (LCG) - simpler RNG with shorter periods and poorer randomness
        - Hardware-based generators - use physical signals (i.e. thermal noise, mouse movements, etc.)
- List real-world applications where MT19937 is used
- Discuss what are the limitations of the MT19937 algorithm.

### B) Generating Random Samples from Continuous Distributions

For the second part, the following tasks are expected to be done:
- Use the algorithm to generate uniform random numbers in [0,1) by drawing samples from the following distributions:
    - Uniform(a, b)
    - Exponential(λ)
    - Normal(μ, σ²)
    - Any other continuous distribution not discussed in class (e.g. Beta, Cauchy, Pareto, etc)
- Explain the mathematical reasons behind the methods used to convert uniform random numbers into the desired distributions (e.g., using inverse transform sampling, or the Box-Muller method for generating normal distribution).
- Plot the histograms vs theoretical PDFs
    - Generate many sampled and plot their histograms
    - Overlay the true PDF curve on the histogram
- Analyze any differences between the generated samples and explain possible reasons for these differences.

## Research on MT19337 Algorithm

### Understanding the Mersenne Twister algorithm

The MT19337 (Mersenne Twister) algorithm is an algorithm developed by Makoto Matsumoto and Takuji Nishimura in 1997 [\[1\]](#References). It is a deterministic pseudorandom number generator (PRNG) which means its output is determined by an initial seed value making sequences reproducible, and it approximates true randomness through mathematical operations (unlike true random generators which depends on physical entropy).

The following diagram shows a simplified version core logic of the MT19337 algorithm.

![MT19937_Simplified_Core_Logic](images/MT19937_Simplified_Core_Logic.png)

From above core logic diagram, we notice the 2 operations that holds the mathematical core of the algorithm. These are the **Twist** and **Temper** operations. Simply put, twisting is done for every 624 generation of a random number, and tempering is done for each output.

**Summary of the Twist Operation:**

For all \( i = 0 \) to \( n-1 \):
\begin{aligned}
y_i &= (\text{MT}_i \land \text{upper\_mask}) + (\text{MT}_{(i+1) \bmod n} \land \text{lower\_mask}) \\
\text{MT}'_i &= \text{MT}_{(i + m) \bmod n} \oplus \left\lfloor \frac{y_i}{2} \right\rfloor \\
\text{if } y_i \bmod 2 = 1 &: \text{MT}'_i \leftarrow \text{MT}'_i \oplus a
\end{aligned}

- _with the following defined constants:_
    | Symbol      | Value         | Description                               |
    |-------------|---------------|-------------------------------------------|
    |   a         | `0x9908B0DF`  | Coefficient for the twist transformation  |
    | upper_mask  | `0x80000000`  | Most significant w-r bits (w=32-bit length of a word)|
    | lower_mask  | `0x7FFFFFFF`  | Least significant r bits                  |

**Summary of the Temper Operation:**

\begin{aligned}
y &\leftarrow y \oplus \left( \frac{y}{2^u} \right) \quad &\text{(right shift)} \\
y &\leftarrow y \oplus \left( (y \ll s) \land b \right) \\
y &\leftarrow y \oplus \left( (y \ll t) \land c \right) \\
y &\leftarrow y \oplus \left( \frac{y}{2^l} \right)
\end{aligned}

- _with the following defined constants:_
    | Symbol | Value         | Description |
    |--------|---------------|-------------|
    |   u    | 11            | Right shift |
    |   s    | 7             | Left shift  |
    |   t    | 15            | Left shift  |
    |   l    | 18            | Right shift |
    |   b    | `0x9D2C5680`  | Bitmask     |
    |   c    | `0xEFC60000`  | Bitmask     |

To gain a better understanding of the algorithm, the sample C code from Matsumoto and Nishimura's paper was studied to generate this sequence diagram.

![MT19337_Detailed_Sequence_Diagram](images/MT19337_Detailed_Sequence_Diagram.png)

### Features of the MT19937 Algorithm

- **has a very long period length**: It can generate a huge number of random numbers (**2^19937-1**) before the sequence starts repeating.
     - This is where the algorithm takes its name from as 2^19937-1 is a Mersenne prime [\[2\]](#references), which is a prime number expressible as:  
         $$
         M_n = 2^n - 1
         $$
         where \(n\) must also be prime.
    - A long period length is important because it prevents repetition patterns that could affect the statistical analysis and simulations. With MT19937's long period, this ensures that repitition is virtually impossible to occur.
    - When comparing the period length of some PRNGs, we can observe that MT19937 is significantly longer:
        | PRNG              | Period Length      |  
        |-------------------|-------------------|  
        | MT19937       | 2^19937 - 1 |  
        | LCG (common implementation) [\[3\]](#references) | 2^32        |  
        | Xorshift [\[4\]](#references)      | 2^128 - 1   |  

- **has high-dimensional uniformity**: It passes statistical tests for uniformity in up to **623 dimensions**. Some of the statistical tests passed that were mentioned in Matsumoto and Nishimura's paper were: 
    - k-distribution test [\[1\]](#references)
    - diehard test [\[5\]](#references)

- **has low memory usage**: Uses a relatively small amount of memory to store its state vector which is only 624 words (156 Bytes).

- **has better efficiency and better performance suitable in certain applications**: MT19937 Algorithm fares to be more efficient than Linear Congruential Generators (LCG) and Hardware RNG. Here's a summary of their efficiency comparison
    - **vs. Linear Congruential Generators (LCG)**  
        | Feature          |   MT19937   |   LCG   |  
        |------------------|------------|--------|  
        | Randomness       | High-dimensional uniformity | Poorer (correlations in higher dimensions) |  
        | Speed            | Slightly slower (due to twist operation) | Faster (single multiply-add step) |  

        _Why MT19937 is Better?_
        - LCGs fail statistical tests in high dimensions, while MT19937 maintains uniformity.

    - **vs. Hardware-Based Generators**
        | Feature          | MT19937 | Hardware RNG |  
        |------------------|------------|-----------------|  
        | Randomness   | Pseudorandom (deterministic) | True randomness (entropy sources) _*example below_ |
        | Speed        | Very fast | Slower (limited by entropy collection) _*explanation provided below_ |
        | Reproducibility | Yes (seed-based) | No |

        _Wall of Lava Lamps: An example of entropy source_
        - One famous example of entropy sources is Cloudflare's wall of Lava lamps [\[6\]](#references). Since the "lava" in lava lamps forms unique patterns that never repeat exactly, monitoring a wall of these lamps provides a great source of randomness for entropy generation.

        _RDRAND: An example of Hardware RNG_
        - Intel's DRNG which calls the RDRAND instruction, which returns random numbers from an Intel on-chip hardware RNG.[\[7\]](#references).
        - The RNG takes pairs of 256-bit raw entropy samples generated by the hardware entropy source. Generating these entropy samples may take some time if there's no natural randomness occurring in the hardware at the moment. For example, an idle PC will take longer to fill-in the entropy samples than that of a PC being actively used where a mouse movement or the thermal signals of the CPU can contribute to the entropy.
        - Basically, what limits and slows down a Hardware RNG is that it involves processes that can't be accelerated beyond physical limits

        _When to Use MT19937 or Hardware RNG_
        - **MT19937**: Applications where reproducibility is needed (i.e. simulations, gaming, machine learning)
        - **Hardware RNG**: Applications where unpredictability is required (i.e. cryptography, security)



#### Real-World Applications

MT19937 is used in:  
1. **Simulations** (Monte Carlo methods, financial modeling).  
2. **Computer Graphics** (procedural noise, random textures).  
3. **Gaming** (randomized loot, procedural generation).  
4. **Programming Languages** (Python, C++, R default PRNG).  
5. **Machine Learning** (random weight initialization).  

#### Limitations of MT19937

Despite its strengths, MT19937 has drawbacks:  
- **Not Cryptographically Secure**: Predictable if enough outputs are observed.  
- **Slow Seeding**: Poor initialization can lead to biased sequences.  
- **Alternatives Exist**: **PCG** and **Xoshiro** offer better speed/quality trade-offs in some cases.  


## References

[1]: Makoto Matsumoto and Takuji Nishimura. 1998. Mersenne twister: a 623-dimensionally equidistributed uniform pseudo-random number generator. ACM Trans. Model. Comput. Simul. 8, 1 (Jan. 1998), 3–30. https://doi.org/10.1145/272991.272995

[2]: "Mersenne prime." Wikipedia, The Free Encyclopedia. https://en.wikipedia.org/wiki/Mersenne_prime

[3]: "Linear congruential generator." Wikipedia, The Free Encyclopedia. https://en.wikipedia.org/wiki/Linear_congruential_generator
 
[4]: "Xorshift." Wikipedia, The Free Encyclopedia. https://en.wikipedia.org/wiki/Xorshift

[5]: "MersenneTwister." CERN Colt Distribution. https://dst.lbl.gov/ACSSoftware/colt/api/cern/jet/random/engine/MersenneTwister.html

[6]: "Lava Lamp Encryption." Cloudflare. https://www.cloudflare.com/learning/ssl/lava-lamp-encryption/

[7]: "Intel® Digital Random Number Generator (DRNG) Software Implementation Guide." Intel. https://www.intel.com/content/www/us/en/developer/articles/guide/intel-digital-random-number-generator-drng-software-implementation-guide.html


